In [31]:
from numpy.core.fromnumeric import mean
from flask import jsonify
import pandas as pd
import numpy as np
import json
import re
import http
import xgboost as xgb
import time
import urllib
import re
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
from itertools import chain, starmap
%load_ext jupyternotify

# years completed 2002-2020
years_arr = [2020]
def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException, http.client.IncompleteRead) as e: 
        print('Download error:', url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
        if num_retries > 0: 
            if e == http.client.IncompleteRead: 
                time.sleep(10)
                return download(url, num_retries - 1)
    return html

def flatten_json_iterative(dictionary, sep = '.', ind_start = 0):
    """Flattening a nested json file"""

    def unpack_one(parent_key, parent_value):
        """Unpack one level (only one) of nesting in json file"""
        # Unpacking one level        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                t1 = parent_key + sep + key
                yield t1, value
        elif isinstance(parent_value, list):
            i = ind_start 
            for value in parent_value:
                t2 = parent_key + sep +str(i) 
                i += 1
                yield t2, value
        else:
            yield parent_key, parent_value    

            
    # Continue iterating the unpack_one function until the terminating condition is satisfied
    while True:
        # Continue unpacking the json file until all values are atomic elements (aka neither a dictionary nor a list)
        dictionary = dict(chain.from_iterable(starmap(unpack_one, dictionary.items())))
        # Terminating condition: none of the values in the json file are a dictionary or a list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

def wnba_pbp(gameId):
        """nba_pbp()
        Pull the game by id
        Data from API endpoints:
        * nba/playbyplay
        * nba/summary
        """
        # play by play
        pbp_url = "http://cdn.espn.com/core/wnba/playbyplay?gameId={}&xhr=1&render=false&userab=18".format(gameId)
        pbp_resp = download(url=pbp_url)
        pbp_txt = {}
        if pbp_resp is not None:
            pbp_txt['teams'] = np.array([])
            pbp_txt['id'] = np.array([])
            pbp_txt['competitions'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['header'] = {}
            pbp_txt['pickcenter'] = np.array([])
            pbp_txt['broadcasts'] = np.array([])
            pbp_txt['videos'] = np.array([])
            pbp_txt['standings'] = np.array([])
            pbp_txt['boxscore'] = np.array([])
            pbp_txt['espnWP'] = np.array([])
            pbp_txt['gameInfo'] = np.array([])
            pbp_txt['season'] = np.array([])
            pbp_txt['timeouts'] = {}
            pbp_d = json.loads(pbp_resp)
            pbp_txt = pbp_d['gamepackageJSON']
            pbp_txt['gameId'] = pbp_d['gameId']
            # summary endpoint for pickcenter array
            summary_url = "http://site.api.espn.com/apis/site/v2/sports/basketball/wnba/summary?event={}".format(gameId)
            summary_resp = download(url=summary_url)
            summary = json.loads(summary_resp)
            if 'pickcenter' in summary.keys():
                summary_txt = summary['pickcenter']
            else:
                summary_txt = {}
                summary_txt['pickcenter']={}
            # ESPN's win probability
            wp = "winprobability"
            if wp in summary:
                espnWP = summary["winprobability"]
            else:
                espnWP = np.array([])

            if 'news' in pbp_txt.keys():
                del pbp_txt['news']
            if 'shop' in pbp_txt.keys():
                del pbp_txt['shop']
            pbp_txt['gameInfo'] = pbp_txt['header']['competitions'][0]
            pbp_txt['season'] = pbp_txt['header']['season']['year']
            pbp_txt['pickcenter'] = summary_txt
            pbp_txt['espnWP'] = espnWP
            # Home and Away identification variables
            homeTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['id'])
            awayTeamId = int(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['id'])
            homeTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['name'])
            awayTeamMascot = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['name'])
            homeTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['location'])
            awayTeamName = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['location'])
            homeTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][0]['team']['abbreviation'])
            awayTeamAbbrev = str(pbp_txt['header']['competitions'][0]['competitors'][1]['team']['abbreviation'])
            homeTeamNameAlt = re.sub("Stat(.+)", "St", str(homeTeamName))
            awayTeamNameAlt = re.sub("Stat(.+)", "St", str(awayTeamName))

            if len(pbp_txt['espnWP']) > 1:
                pbp_txt['espnWP'] = espnWP
            else:
                pbp_txt['espnWP'] = espnWP
            pbp_txt['plays_mod'] = []
            for play in pbp_txt['plays']:
                p = flatten_json_iterative(play)
                pbp_txt['plays_mod'].append(p)
            pbp_txt['plays'] = pd.json_normalize(pbp_txt,'plays_mod')
            if len(pbp_txt['plays'])>1:
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                # Spread definition
                pbp_txt['plays']["homeTeamSpread"] = 2.5
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                pbp_txt['plays'] = pd.DataFrame(pbp_txt['plays'])
                pbp_txt['plays']['season'] = pbp_txt['header']['season']['year']
                pbp_txt['plays']['seasonType'] = pbp_txt['header']['season']['type']
                pbp_txt['plays']["homeTeamId"] = homeTeamId
                pbp_txt['plays']["awayTeamId"] = awayTeamId
                pbp_txt['plays']["homeTeamName"] = str(homeTeamName)
                pbp_txt['plays']["awayTeamName"] = str(awayTeamName)
                pbp_txt['plays']["homeTeamMascot"] = str(homeTeamMascot)
                pbp_txt['plays']["awayTeamMascot"] = str(awayTeamMascot)
                pbp_txt['plays']["homeTeamAbbrev"] = str(homeTeamAbbrev)
                pbp_txt['plays']["awayTeamAbbrev"] = str(awayTeamAbbrev)
                pbp_txt['plays']["homeTeamNameAlt"] = str(homeTeamNameAlt)
                pbp_txt['plays']["awayTeamNameAlt"] = str(awayTeamNameAlt)
                pbp_txt['plays']['period.number'] = pbp_txt['plays']['period.number'].apply(lambda x: int(x))
                #----- Figuring out Timeouts ---------
                pbp_txt['timeouts'] = {}
                pbp_txt['timeouts'][homeTeamId] = {"1": [], "2": []}
                pbp_txt['timeouts'][awayTeamId] = {"1": [], "2": []}
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["gameSpread"] = abs(gameSpread)
                pbp_txt['plays']["homeTeamSpread"] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
                pbp_txt['plays']["homeFavorite"] = homeFavorite
                pbp_txt['plays']["gameSpread"] = gameSpread
                pbp_txt['plays']["homeFavorite"] = homeFavorite

                #----- Time ---------------
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.displayValue'].str.split(pat=':')
                pbp_txt['plays']['clock.mm'] = pbp_txt['plays']['clock.mm'].apply(lambda m: ['0'] + m if len(m) < 2 else m)
                pbp_txt['plays'][['clock.minutes','clock.seconds']] = pbp_txt['plays']['clock.mm'].to_list()
                pbp_txt['plays']['half'] = np.where(pbp_txt['plays']['period.number'] <= 2, "1","2")
                pbp_txt['plays']['lag_half'] = pbp_txt['plays']['half'].shift(1)
                pbp_txt['plays']['lead_half'] = pbp_txt['plays']['half'].shift(-1)

                pbp_txt['plays']['game_play_number'] = np.arange(len(pbp_txt['plays']))+1
                pbp_txt['plays']['text'] = pbp_txt['plays']['text'].astype(str)
                pbp_txt['plays']['id'] = pbp_txt['plays']['id'].apply(lambda x: int(x))
                del pbp_txt['plays']['clock.mm']
            else:
                if len(pbp_txt['pickcenter']) > 1:
                    if 'spread' in pbp_txt['pickcenter'][1].keys():
                        gameSpread =  pbp_txt['pickcenter'][1]['spread']
                        homeFavorite = pbp_txt['pickcenter'][1]['homeTeamOdds']['favorite']
                    else:
                        gameSpread =  pbp_txt['pickcenter'][0]['spread']
                        homeFavorite = pbp_txt['pickcenter'][0]['homeTeamOdds']['favorite']

                else:
                    gameSpread = 2.5
                    homeFavorite = True
                pbp_txt['homeTeamSpread'] = np.where(homeFavorite == True, abs(gameSpread), -1*abs(gameSpread))
            
            
            pbp_json = {
                "gameId": pbp_txt['gameId'],
                "plays" : pbp_txt['plays'].to_dict(orient='records'),
                "winprobability" : np.array(pbp_txt['winprobability']).tolist(),
                "boxscore" : pbp_txt['boxscore'],
                "header" : pbp_txt['header'],
                "homeTeamSpread" : np.array(pbp_txt['homeTeamSpread']).tolist(),
                "broadcasts" : np.array(pbp_txt['broadcasts']).tolist(),
                "videos" : np.array(pbp_txt['videos']).tolist(),
                "standings" : pbp_txt['standings'],
                "pickcenter" : np.array(pbp_txt['pickcenter']).tolist(),
                "espnWP" : np.array(pbp_txt['espnWP']).tolist(),
                "gameInfo" : np.array(pbp_txt['gameInfo']).tolist(),
                "season" : np.array(pbp_txt['season']).tolist()
            }
            return pbp_json, pbp_json['season']
        else:
            pass
        
    
schedule = pd.read_csv('wnba_games_info_2002_2021.csv', encoding='latin-1')
schedule = schedule.sort_values(by=['season', 'date'], ascending = False)

games = schedule[(schedule['season'].isin(years_arr))&(schedule['status.type.name']=='STATUS_FINAL')].reset_index()['id']
print(f"Number of Games: {len(games)}, first gameId: {games[0]}")
g, y = wnba_pbp(gameId = games[0])
# for key in g.keys():
#     print(f"{key}: {type(g[key])}")    
json.dumps(g,indent=4)
print(g.keys())
print(pd.DataFrame(g['plays']))
y

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify
Number of Games: 147, first gameId: 401244185
dict_keys(['gameId', 'plays', 'winprobability', 'boxscore', 'header', 'homeTeamSpread', 'broadcasts', 'videos', 'standings', 'pickcenter', 'espnWP', 'gameInfo', 'season'])
     shootingPlay sequenceNumber period.displayValue  period.number  \
0           False              4         1st Quarter              1   
1           False              7         1st Quarter              1   
2           False              9         1st Quarter              1   
3            True             10         1st Quarter              1   
4           False             11         1st Quarter              1   
..            ...            ...                 ...            ...   
383          True            536         4th Quarter              4   
384         False            537         4th Quarter              4   
385          True            538         4th Quar

2020

In [32]:
%%notify
# 2020:
# 2019: 
# 2018:
# 2017:
# 2016:
# 2015:
# 2014:
# 2013:
# 2012:
# 2011:
# 2010:
# 2009:
# 2008:
# 2007:
# 2006: 
# 2005:
# 2004:
# 2003:
# 2002:
i = 0
for game in games[i:]:
    print(f"Working on game {i+1} of {len(games)}, gameId: {games[i]}")
    if len(str(game))<9:
        i+=1
        continue
    try:
        g, y = wnba_pbp(gameId=game)
    except (TypeError) as e: 
        print("TypeError: yo", e)
        g, y = wnba_pbp(gameId=game)
        
    except (KeyError) as e:
        print("KeyError: yo", e)
        i+=1
        continue
    except (ValueError) as e:
        print("DecodeError: yo", e)
        i+=1
        continue
    fp = "wnba/{}/{}.json".format(y, game)
    with open(fp,'w') as f:
        json.dump(g, f, indent=4, sort_keys=False) 
#     time.sleep(1)        
    i+=1

<IPython.core.display.Javascript object>